<h1>Setup Postgres</h1>
We install postgres and its dev tools (necessary to build lantern from source). We also start postgres, and set up a user 'postgres' with password 'postgres' and create a database called 'ourdb'

In [19]:
# We install postgres and its dev tools
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql postgresql-server-dev-all
#  Start postgres
!sudo service postgresql start

# Create user, password, and db
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS testdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE testdb;'

 * Starting PostgreSQL 14 database server
   ...done.
ALTER ROLE
ERROR:  database "testdb" is being accessed by other users
DETAIL:  There is 1 other session using the database.
ERROR:  database "testdb" already exists


In [20]:
!git clone --recursive https://github.com/lanterndata/lantern.git
%cd lantern
!mkdir build
%cd build
!pwd
!cmake ..
!make install

Cloning into 'lantern'...
remote: Enumerating objects: 2689, done.
remote: Counting objects: 100% (1463/1463), done.
remote: Compressing objects: 100% (582/582), done.
remote: Total 2689 (delta 1105), reused 990 (delta 875), pack-reused 1226
Receiving objects: 100% (2689/2689), 646.21 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (1745/1745), done.
Submodule 'third_party/hnswlib' (https://github.com/ngalstyan4/hnswlib) registered for path 'third_party/hnswlib'
Submodule 'third_party/usearch' (https://github.com/ngalstyan4/usearch) registered for path 'third_party/usearch'
Cloning into '/content/lantern/build/lantern/third_party/hnswlib'...
remote: Enumerating objects: 1723, done.        
remote: Counting objects: 100% (343/343), done.        
remote: Compressing objects: 100% (43/43), done.        
remote: Total 1723 (delta 314), reused 300 (delta 300), pack-reused 1380        
Receiving objects: 100% (1723/1723), 528.17 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (1096/1096), done

In [21]:
!pip install -qU \
  datasets==2.12.0 \
  sentence-transformers==2.2.2

In [22]:
import re
import requests
import xml.etree.ElementTree as ET

def get_pmc_paper(pmcid):
    url = f'https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML'
    req = requests.get(url)
    res = req.text
    return res

def get_sentence_from_text(text):
    return text.split(".")

In [23]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [24]:

import psycopg2

# We use the dbname, user, and password that we specified above
conn = psycopg2.connect(
    dbname="testdb",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432" # default port for Postgres
)

# Get a new cursor
cursor = conn.cursor()

# Execute the query to load the Lantern extension in
cursor.execute("CREATE EXTENSION IF NOT EXISTS lantern;")

conn.commit()
cursor.close()

In [25]:

# Create the table
cursor = conn.cursor()

create_table_query = "CREATE TABLE fragments (id serial PRIMARY key, content text, vector real[]);"

cursor.execute(create_table_query)

conn.commit()
cursor.close()

In [26]:
## Write to file
# pmcid = 'PMC9935389'
pmcid = 'PMC10081221'
text = get_pmc_paper(pmcid)
sentences = get_sentence_from_text(text)

In [27]:

from tqdm.auto import tqdm

cursor = conn.cursor()

for i in tqdm(range(0, len(sentences))):
    content = sentences[i]

    # Create embedding for the question
    vector = [float(x) for x in model.encode(sentences[i])]

    # Insert the content of the question as well as the embedding into our db
    cursor.execute("INSERT INTO fragments (content, vector) VALUES (%s, %s);", (content, vector))

conn.commit()
cursor.close()


  0%|          | 0/938 [00:00<?, ?it/s]

In [28]:

cursor = conn.cursor()

cursor.execute("CREATE INDEX ON fragments USING hnsw (vector dist_cos_ops) WITH (dim=384);")

conn.commit()
cursor.close()


In [29]:
cursor = conn.cursor()

cursor.execute("SELECT * FROM fragments;")
print(cursor.fetchone())

conn.commit()
cursor.close()

(1, '<!DOCTYPE article PUBLIC "-//NLM//DTD JATS (Z39', [-0.10038493, 0.074392885, -0.010277012, 0.03291725, 0.06306493, -0.00015471199, -0.0032982235, 0.08994667, 0.04761151, 0.022519236, 0.06836179, 0.06972122, 0.04632185, 0.010354124, 0.01586501, -0.0033510588, -0.07219918, 0.02590406, 0.023794778, 0.011029444, -0.0017973067, 0.12902358, 0.07414243, -0.00579261, 0.033352446, 0.039338063, -0.032031316, -0.03647587, 0.040215693, -0.03105971, -0.0213532, 0.06690294, 0.016746547, 0.039464366, 0.031025302, -0.018622361, 0.06215862, -0.051095854, 0.024625102, 0.05517938, 0.0358864, -0.0035000104, -0.017162679, -0.030135475, -0.0003620154, 0.02771113, -0.08590952, -0.048629884, -0.10256854, 0.08413096, -0.07823631, -0.018038584, 0.057737686, 0.012949463, 0.05738016, -0.016432116, -0.015333189, 0.044870086, -0.04882088, -0.011113188, 0.019751418, 0.0507649, -0.120700456, 0.033393655, 0.057415105, -0.0036551345, 0.023633579, 0.018545281, -0.044210013, -0.039641812, -0.029899692, -0.019151194,

In [30]:
!sudo -u postgres pg_dump testdb > out.sql